# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install openai==2.11.0 gradio==6.1.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. LangChain V1.0 ReAct Agent 搭建

## 2.1 简介

在最新的 LangChain V1.0 版本中，LangChain 官方已经全面将 Agent 组件中的相关内容接入到 LangGraph 中。LangGraph 是 LangChain 团队在 2024 年推出的新一代框架，用于构建具备“状态、记忆与多步推理能力”的智能体（Agent）。

它以“图（Graph）”为核心思想：把每个模型调用、工具执行、判断逻辑都视为一个节点，通过节点间的有向边来定义信息流动，从而让复杂的智能体流程（如 ReAct、对话管理、多智能体协作、人工中断等）都能用图结构清晰地表达。

相比旧版 LangChain 的线性链式调用，LangGraph 提供了可控、可追踪、可持久化的执行流，支持状态保存、检查点（Checkpoint）、时间回溯（Time Travel）等功能，是目前官方推荐的 Agent 开发核心框架。

## 2.2 LLM 模型

由于在 LangGraph 中，需要大模型支持工具调用（Function Calling）。因为通义千问明确在 LangChain 文档中表示支持，因此后续我们将使用 Qwen 系列模型进行演示。

In [3]:
from langchain_community.chat_models import ChatTongyi
import os

llm = ChatTongyi(
  api_key=os.environ.get("DASHSCOPE_API_KEY"), 
  model="qwen-max")

response = llm.invoke("你好，请介绍一下你自己")

print(response.content)

你好！我是Qwen，是阿里云开发的一款超大规模语言模型。我的设计目标是成为一款能够理解、学习并生成接近人类水平的文本的AI助手。我可以帮助用户完成多种多样的任务，比如撰写文章、创作故事、写邮件、表达观点、玩游戏等。同时，我也在不断学习和进步中，希望能够更好地服务于大家。如果你有任何问题或需要帮助的地方，请随时告诉我哦！


## 2.3 Memory 记忆

在新版本里，我们不再需要通过 RunnableWithMessageHistory 的方式进行记忆的保留，在 LangGraph 下我们使用 InMemorySaver() 的方式进行保留。

所谓的短期记忆（Short-term Memory），实际上指的是系统仅在当前对话轮次中保存的临时上下文信息，用于维持一次连续的对话逻辑或局部推理过程。

在 LangGraph 中，InMemorySaver() 的作用就是在智能体（Agent）或图（Graph）执行的过程中，临时保存模型的输入输出、对话内容、工具调用记录以及节点状态等信息，从而使系统在一次会话（session）中能够“记住”先前的交互历史。

In [4]:
from langgraph.checkpoint.memory import InMemorySaver 
memory = InMemorySaver()

## 2.4 System Prompt 系统提示词

我们不需要再构建一大段的提示词了，而是只需要描述能力即可。

In [5]:
system_prompt = "You are a helpful assistant"

## 2.5 Tool 工具

在工具的定义中，新版本并未作出太大的改变，我们还是可以使用之前工具定义的函数进行执行。

In [6]:
from langchain.tools import tool

@tool
def calculate(what: str) -> str:
  """
  calculate:
  e.g. calculate: 4 * 7 / 3
  Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
  """
  return str(eval(what))

@tool
def average_dog_weight(name: str) -> str:
  """
  average_dog_weight:
  e.g. average_dog_weight: Collie
  returns average weight of a dog when given the breed
  """
  name = name.lower()
  if "scottish terrier" in name:
    return "Scottish Terriers average 20 lbs"
  elif "border collie" in name:
    return "A Border Collie's average weight is 37 lbs"
  elif "toy poodle" in name:
    return "A Toy Poodle's average weight is 7 lbs"
  else:
    return "An average dog weighs 50 lbs"
  
tools = [calculate, average_dog_weight]

## 2.6 系统组装

在准备好了一些基础的组件以后，我们使用 create_agent 的方法将这些内容组合起来：

In [7]:
from langchain.agents import create_agent

agent = create_agent(model=llm, 
           tools=tools, 
           system_prompt=system_prompt, 
           checkpointer=memory)

In [8]:
result1 = agent.invoke({"messages": [{"role": "user", "content": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?"}]}, config={"configurable": {"thread_id": "user_1"}})
print(result1)

{'messages': [HumanMessage(content='I have 2 dogs, a border collie and a scottish terrier. What is their combined weight? Could you double it?', additional_kwargs={}, response_metadata={}, id='d6fd07e6-a683-4ed0-bd5a-b75a9dc98d82'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"name": "border collie"}', 'name': 'average_dog_weight'}, 'id': 'call_9956e6334f194da98f519f', 'index': 0, 'type': 'function'}, {'function': {'arguments': '{"name": "scottish terrier"}', 'name': 'average_dog_weight'}, 'id': 'call_09d50ce3085e4312a63c79', 'index': 1, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '35614fc0-5066-48cc-acbf-e427767903cb', 'token_usage': {'input_tokens': 369, 'output_tokens': 47, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 416}}, id='lc_run--019b2fbd-a677-7440-b684-7e53313b2557-0', tool_calls=[{'name': 'average_dog_weight', 'args': {'name': 'border collie'}, 'id

假如只需要答案的话：

In [9]:
print(result1["messages"][-1].content)

The doubled combined weight of a Border Collie and a Scottish Terrier is 114 lbs.


In [10]:
import gradio as gr
from langchain.agents import create_agent

agent = create_agent(model=llm, 
        tools=tools, 
        system_prompt=system_prompt, 
        checkpointer=memory)

def agent_response(content, history):
    result1 = agent.invoke({"messages": [{"role": "user", "content": content}]}, config={"configurable": {"thread_id": "user_1"}})
    return result1["messages"][-1].content

demo = gr.ChatInterface(fn=agent_response)

demo.launch()

c:\Users\76391\.conda\envs\chapter_5_2025_12_15\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
